In [ ]:
!pip install watermark

In [2]:
%load_ext watermark

In [3]:
%watermark -a "Rishit Kapoor 21BAI1046" -u -d -v -m

Author: Rishit Kapoor 21BAI1046

Last updated: 2024-04-13

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [ ]:
!pip install kaggle

In [2]:
!mkdir ~/.kaggle

In [3]:
!cp kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download muthuj7/weather-dataset

 90% 2.00M/2.23M [00:01<00:00, 2.20MB/s]
100% 2.23M/2.23M [00:01<00:00, 2.10MB/s]


In [6]:
!unzip '/content/weather-dataset.zip' -d '/content/weather-dataset'

Archive:  /content/weather-dataset.zip
  inflating: /content/weather-dataset/weatherHistory.csv  


In [7]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the modified split_sequence function
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Load the data
data = pd.read_csv("/content/weather-dataset/weatherHistory.csv")

# Select only the 'Temperature (C)' column
temperature_data = data['Temperature (C)'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature_data)

# Define the number of time steps
n_steps = 10

# Split the temperature sequence into input/output
X, y = split_sequence(scaled_data, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=32)

# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)

# Inverse transform the predicted temperature
predicted_temperature = scaler.inverse_transform(predicted_temperature)


Epoch 1/100
3014/3014 [==============================] - 23s 6ms/step - loss: 0.0023
Epoch 2/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.8827e-04
Epoch 3/100
3014/3014 [==============================] - 18s 6ms/step - loss: 5.5905e-04
Epoch 4/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.4896e-04
Epoch 5/100
3014/3014 [==============================] - 18s 6ms/step - loss: 5.3825e-04
Epoch 6/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.3293e-04
Epoch 7/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.2974e-04
Epoch 8/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.2452e-04
Epoch 9/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.1995e-04
Epoch 10/100
3014/3014 [==============================] - 19s 6ms/step - loss: 5.1514e-04
Epoch 11/100
3014/3014 [==============================] - 18s 6ms/step - loss: 5.1079e-04
Epoch 12/100
3014/3014 

In [9]:
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)

1/1 [==============================] - 0s 34ms/step


In [10]:
print(last_sequence.shape)

(1, 10, 1)


In [17]:
# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)

# Print predicted_temperature
print("Predicted Temperature Array:", predicted_temperature)

# Convert the prediction array to a scalar
predicted_temperature_scalar = np.squeeze(predicted_temperature)

# Inverse transform the predicted temperature scalar
predicted_temperature_scalar = scaler.inverse_transform([[predicted_temperature_scalar]])

# Compute evaluation metrics
y_true = temperature_data[-1]
y_pred = predicted_temperature_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Temperature (C):", y_true)
print("Predicted Temperature (C):", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 0s 18ms/step
Predicted Temperature Array: [[0.6613921]]
Sample Prediction Details:
True Temperature (C): [20.43888889]
Predicted Temperature (C): [19.0040419]
Mean Absolute Error (MAE): 1.434846988651497
Root Mean Squared Error (RMSE): 1.434846988651497
Mean Absolute Percentage Error (MAPE): 7.020180971929041


In [19]:
# Stacked Lstm

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Define the modified split_sequence function
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Load the data
data = pd.read_csv("/content/weather-dataset/weatherHistory.csv")

# Select only the 'Temperature (C)' column
temperature_data = data['Temperature (C)'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature_data)

# Define the number of time steps
n_steps = 10

# Split the temperature sequence into input/output
X, y = split_sequence(scaled_data, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the LSTM model with stacked layers
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(units=50, return_sequences=True))  # Stacked LSTM layer
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=64)

# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)

# Inverse transform the predicted temperature
predicted_temperature = scaler.inverse_transform(predicted_temperature)


Epoch 1/100
1507/1507 [==============================] - 17s 8ms/step - loss: 0.0043
Epoch 2/100
1507/1507 [==============================] - 12s 8ms/step - loss: 6.5328e-04
Epoch 3/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.8602e-04
Epoch 4/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.6525e-04
Epoch 5/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.5472e-04
Epoch 6/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.5079e-04
Epoch 7/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.4634e-04
Epoch 8/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.3389e-04
Epoch 9/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.3237e-04
Epoch 10/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.2016e-04
Epoch 11/100
1507/1507 [==============================] - 12s 8ms/step - loss: 5.2085e-04
Epoch 12/100
1507/1507 

In [20]:
# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)


# Convert the prediction array to a scalar
predicted_temperature_scalar = np.squeeze(predicted_temperature)

# Inverse transform the predicted temperature scalar
predicted_temperature_scalar = scaler.inverse_transform([[predicted_temperature_scalar]])

# Compute evaluation metrics
y_true = temperature_data[-1]
y_pred = predicted_temperature_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Temperature (C):", y_true)
print("Predicted Temperature (C):", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 0s 18ms/step
Sample Prediction Details:
True Temperature (C): [20.43888889]
Predicted Temperature (C): [19.16473]
Mean Absolute Error (MAE): 1.2741588883929857
Root Mean Squared Error (RMSE): 1.2741588883929857
Mean Absolute Percentage Error (MAPE): 6.233992930435917


In [21]:
#univariate Bidirectional LSTM

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import math
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional

# Define the modified split_sequence function
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Load the data
data = pd.read_csv("/content/weather-dataset/weatherHistory.csv")

# Select only the 'Temperature (C)' column
temperature_data = data['Temperature (C)'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(temperature_data)

# Define the number of time steps
n_steps = 10

# Split the temperature sequence into input/output
X, y = split_sequence(scaled_data, n_steps)

# Reshape data for LSTM [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# Define the Bidirectional LSTM model
model = Sequential()
model.add(Bidirectional(LSTM(units=50, input_shape=(n_steps, 1))))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X, y, epochs=100, batch_size=64)

# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)

# Inverse transform the predicted temperature
predicted_temperature = scaler.inverse_transform(predicted_temperature)


Epoch 1/100
1507/1507 [==============================] - 16s 8ms/step - loss: 0.0027
Epoch 2/100
1507/1507 [==============================] - 17s 11ms/step - loss: 7.1565e-04
Epoch 3/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.9763e-04
Epoch 4/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.6949e-04
Epoch 5/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.5828e-04
Epoch 6/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.4620e-04
Epoch 7/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.4575e-04
Epoch 8/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.3823e-04
Epoch 9/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.3366e-04
Epoch 10/100
1507/1507 [==============================] - 9s 6ms/step - loss: 5.2675e-04
Epoch 11/100
1507/1507 [==============================] - 10s 6ms/step - loss: 5.2539e-04
Epoch 12/100
1507/1507 [======

In [22]:
# Predict on the last time step of the data
last_sequence = scaled_data[-n_steps:]
last_sequence = last_sequence.reshape((1, n_steps, 1))
predicted_temperature = model.predict(last_sequence)


# Convert the prediction array to a scalar
predicted_temperature_scalar = np.squeeze(predicted_temperature)

# Inverse transform the predicted temperature scalar
predicted_temperature_scalar = scaler.inverse_transform([[predicted_temperature_scalar]])

# Compute evaluation metrics
y_true = temperature_data[-1]
y_pred = predicted_temperature_scalar[-1]
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(mean_squared_error(y_true, y_pred))
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

print("Sample Prediction Details:")
print("True Temperature (C):", y_true)
print("Predicted Temperature (C):", y_pred)
print("Mean Absolute Error (MAE):", mae)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Percentage Error (MAPE):", mape)


1/1 [==============================] - 0s 19ms/step
Sample Prediction Details:
True Temperature (C): [20.43888889]
Predicted Temperature (C): [18.95640649]
Mean Absolute Error (MAE): 1.4824823972251764
Root Mean Squared Error (RMSE): 1.4824823972251764
Mean Absolute Percentage Error (MAPE): 7.253243585227826
